## CA1 Search Algorithms
## Arya Fayyazi (810198453)
In this project we are going to find an optimal path with BFS , IDS , A* algorithms to the final state when we put all the ring partners to their goal positions.

In [64]:
from collections import deque
from copy import copy
from math import sqrt
import time
file_1 = open("test_02.txt", "r")
actions = ["up", "right", "left", "down"]
lines = file_1.readlines()
n, m = lines[0].split(" ")
m = m[:-1]
x0, y0 = lines[1].split(" ")
y0 = y0[:-1]
xn, yn = lines[2].split(" ")
yn = yn[:-1]
k, l = lines[3].split(" ")
l = l[:-1]
Orks_pos = []
Ring_Partners = []
Partners_Final = []
for i in range(int(k)):
    x, y , c = lines[i + 4].split(" ")
    c = c[:-1]
    Orks_pos.append([int(x), int(y) , int(c)])
for i in range(int(l)):
    x, y = lines[i + int(k) + 4].split(" ")
    y = y[:-1]
    Ring_Partners.append([int(x), int(y)])
for i in range(int(l)):
    x, y = lines[int(k) + int(l) + i + 4].split(" ")
    Partners_Final.append([int(x), int(y)])
table = [[['.']] * int(m) for i in range(int(n))]
ID1 = 0
ID = 0
ID2 = 0
for i in range(len(Ring_Partners)):
    table[Ring_Partners[i][0]][Ring_Partners[i][1]] = ['RPP',ID1]
    ID1 += 1
for i in range(len(Partners_Final)):
    table[Partners_Final[i][0]][Partners_Final[i][1]] = ['RPFP',ID2]
    ID2 += 1
for i in range(len(Orks_pos)):
    for j in range(max(0,Orks_pos[i][0]-Orks_pos[i][2]), min((int(m)-1,Orks_pos[i][0]+Orks_pos[i][2]))+1) :
        for p in range(max(0,Orks_pos[i][1]-Orks_pos[i][2]), min((int(n)-1,Orks_pos[i][1]+Orks_pos[i][2]))+1) :
            if abs(Orks_pos[i][0]- j) + abs(Orks_pos[i][1]- p) <= Orks_pos[i][2] :
                table[j][p] = [Orks_pos[i][2],ID]
    table[Orks_pos[i][0]][Orks_pos[i][1]] = ['OP']
    ID += 1
for i in range(len(table)):
    print(table[i])


[['.'], ['.'], ['.'], ['.'], ['.'], ['.'], ['.'], ['RPP', 0]]
[['.'], [2, 0], ['.'], ['.'], ['.'], ['.'], [1, 1], ['.']]
[[2, 0], [2, 0], [2, 0], ['.'], ['.'], [1, 1], ['OP'], [1, 1]]
[[2, 0], ['OP'], [2, 0], [2, 0], ['.'], ['.'], [1, 1], ['.']]
[[2, 0], [2, 0], [2, 0], ['.'], [2, 2], ['.'], ['.'], ['.']]
[['RPFP', 0], [2, 0], ['.'], [2, 2], [2, 2], [2, 2], ['RPFP', 1], ['.']]
[['.'], ['.'], [2, 2], [2, 2], ['OP'], [2, 2], [2, 2], ['.']]
[['.'], ['RPP', 1], ['.'], [2, 2], [2, 2], [2, 2], ['.'], ['.']]


In the cell bellow state class is defined which represent each state of this problem .

Each object of this class has some attributes such as :

locations_gan : storing Gandolf locations in map

location_partner : storing Partners locations in map

distances : number of moves that Gandolf has made from the first state

already_in_danger : its True or False that show us we are currently in orks ranges or not.

max_danger : the number of moves we can do in the current state in orks ranges.

goal_pos :  a list of 0 and 1 that show us which partners are arrived to their goal positions.

id : each object of this class is assigned with an id so that they can be distinguished from each other if needed

picked : its True or False that show us Gandolf has any partner with him currentl or not.

ork_id : show the orks ids

partner_picked : show the partner that currently picked id

 Some of the important methods:

represent_as_str : this method returns a string for an each state that contains by order : the location of Gandolf , Gandolf has any partner with him or not , the id of the partner that is with Gandolf and how many of the partners are already reach their goal positions in this state . this string is used for  keys of the visited dictionary.

represent_as_str_with_id : this method is very similar to the above method but the only diffrence is that in this methode at the end of the string we have the id of the state we are on it . this string is used for key and values of father dictionary which this dictionary is used for printng the solution path .

represent_as_str_with_cost : this method is also very similar to represent_as_str_with_id but the only diffrence is at the end of this string we have the cost(g(n)+alpha*h(n)) of the state we are on it instead of id.

In [65]:
class State:
    Id = 0
    def __init__(self,location_gan,location_partner,already_in_danger,max_danger,goal_pos,picked,partner_picked,ork_id):
        self.location_gan = copy(location_gan)
        self.location_partner = copy(location_partner)
        self.distance = 0
        self.already_in_danger = already_in_danger
        self.max_danger = max_danger
        self.goal_pos = copy(goal_pos)
        self.picked = picked
        self.ork_id = ork_id
        self.partner_picked = partner_picked
        self.id = State.Id
        State.Id += 1
    
    def __lt__(self, other):
        if (self.id < other.id):
            return True
        else:
            return False
    def get_row(self):
        return self.location_gan[0]
    def get_column(self):
        return self.location_gan[1]
    def get_already_in_danger(self):
        return self.already_in_danger
    def get_max_danger(self):
        return self.max_danger
    def get_goal_pos(self):
        return self.goal_pos
    def get_id(self):
        return self.id
    def get_loc_gan(self):
        return self.location_gan
    def get_loc_part(self):
        return self.location_partner
    def get_picked(self):
        return self.picked
    def get_ork_id(self):
        return self.ork_id
    def get_partner_picked(self):
        return self.partner_picked
    def set_picked(self,picked):
        self.picked = picked
    def set_partner_picked(self,partner_picked):
        self.partner_picked = partner_picked
    def represent_as_str(self):
        r_str = ""
        r_str += str(self.location_gan[0]) + str(self.location_gan[1])
        if self.picked :
            r_str += "1"
        else:
            r_str += "0"
        r_str += str(self.partner_picked)
        for goal in self.goal_pos:
            r_str += str(goal)
        return r_str
    def finish_condition(self):
        if [self.location_gan[0], self.location_gan[1]] == [int(xn) , int(yn)]:
            if sum(self.goal_pos) == int(l):
                return True
        return False
    def represent_as_str_with_id(self):
        id_str = self.represent_as_str()
        id_str += "-"
        id_str += str(self.id)
        return id_str
    def set_remained_depth(self,depth):
        self.remained_depth = depth
    def get_remained_depth(self):
        return self.remained_depth
    def set_distance(self,distance):
        self.distance = distance
    def get_distance(self):
        return self.distance 
    def get_manhattan_distance(self):
        sum_of_manhattans=0
        result_m = sqrt(((int(n)-1) - self.location_gan[0])**2 + ((int(m)-1)- self.location_gan[1])**2)
        result_y = abs(self.location_gan[1] - (int(m)-1))
        # if self.picked :
        #     result_x = abs(self.location_gan[0] - (self.location_partner[self.partner_picked][0]))
        #     result_y = abs(self.location_gan[1] - (self.location_partner[self.partner_picked][1])) 
        sum_of_manhattans += (result_m)
        
        return sum_of_manhattans
    def represent_as_str_with_cost(self , alpha):
        heuristic_str = self.represent_as_str()
        heuristic_str += "-"
        cost=0
        cost += (self.get_distance() + (alpha * self.get_manhattan_distance()))
        heuristic_str += str(cost)
        return heuristic_str

    


 First state is defined in cell bellow as an object from above class which it's locations is [[x0,y0]].

In [66]:
already_in_danger = False
max_danger = 0
if table[int(x0)][int(y0)][0] != '.' and table[int(x0)][int(y0)][0] != 'RPP' and table[int(x0)][int(y0)][0] != 'RPFP' and table[int(x0)][int(y0)][0] != 'OP' :
    already_in_danger = True
    max_danger = table[int(x0)][int(y0)][0]
first_state = State([int(x0), int(y0)], Ring_Partners, already_in_danger, max_danger,[0] * int(l),False,0,0)
father = {}


In the cell below we have the most two important functions that we use in all of the algorithm. the first one is allowable_action and show us which actions are allowable and the second one is transition_func which make the child_state.

In [67]:

def allowable_action(cur_state, action):
    # table[i][j] == "OP" means that table[i][j] is blocked and we cant go there.

    if action == "up" and cur_state.get_row() < int(n) - 1 and table[cur_state.get_row() + 1][
        cur_state.get_column()][0] != 'OP':
        if cur_state.get_already_in_danger() == False or (cur_state.get_already_in_danger() == True and cur_state.get_max_danger() > 0) or (cur_state.get_already_in_danger() == True
         and cur_state.get_max_danger() == 0 and (table[cur_state.get_row() + 1][cur_state.get_column()][0] == '.' or table[cur_state.get_row() + 1][cur_state.get_column()][0] == 'RPP' 
         or table[cur_state.get_row() + 1][cur_state.get_column()][0] == 'RPFP' or table[cur_state.get_row() + 1][cur_state.get_column()][1] != table[cur_state.get_row()][cur_state.get_column()][1])):
            return True
    elif action == "down" and cur_state.get_row() >= 1 and table[cur_state.get_row() - 1][
        cur_state.get_column()][0] != 'OP':
        if cur_state.get_already_in_danger() == False or (cur_state.get_already_in_danger() == True and cur_state.get_max_danger() > 0) or (cur_state.get_already_in_danger() == True
         and cur_state.get_max_danger() == 0 and (table[cur_state.get_row() - 1][cur_state.get_column()][0] == '.' or table[cur_state.get_row() - 1][cur_state.get_column()][0] == 'RPP' 
         or table[cur_state.get_row() - 1][cur_state.get_column()][0] == 'RPFP' or table[cur_state.get_row() - 1][cur_state.get_column()][1] != table[cur_state.get_row()][cur_state.get_column()][1])):
            return True
    elif action == "left" and cur_state.get_column() >= 1 and table[cur_state.get_row()][
        cur_state.get_column() - 1][0] != 'OP':
        if cur_state.get_already_in_danger() == False or (cur_state.get_already_in_danger() == True and cur_state.get_max_danger() > 0) or (cur_state.get_already_in_danger() == True
         and cur_state.get_max_danger() == 0 and (table[cur_state.get_row()][cur_state.get_column() - 1][0] == '.' or table[cur_state.get_row()][cur_state.get_column() - 1][0] == 'RPP' 
         or table[cur_state.get_row()][cur_state.get_column() - 1][0] == 'RPFP' or table[cur_state.get_row()][cur_state.get_column() - 1][1] != table[cur_state.get_row()][cur_state.get_column()][1])):
            return True
    elif action == "right" and cur_state.get_column() < int(m) - 1 and table[cur_state.get_row()][
        cur_state.get_column() + 1][0] != 'OP':
        if cur_state.get_already_in_danger() == False or (cur_state.get_already_in_danger() == True and cur_state.get_max_danger() > 0) or (cur_state.get_already_in_danger() == True
         and cur_state.get_max_danger() == 0 and (table[cur_state.get_row()][cur_state.get_column() + 1][0] == '.' or table[cur_state.get_row()][cur_state.get_column() + 1][0] == 'RPP' 
         or table[cur_state.get_row()][cur_state.get_column() + 1][0] == 'RPFP' or table[cur_state.get_row()][cur_state.get_column() + 1][1] != table[cur_state.get_row()][cur_state.get_column()][1])):
            return True
    else:
        return False


def transition_func(cur_state, action):
    new_state_location_gandolf = copy(cur_state.get_loc_gan())
    new_state_location_partners = copy(cur_state.get_loc_part())
    new_state_goal_pos = copy(cur_state.get_goal_pos())

    already_in_danger = False
    max_danger = 0
    picked = False
    partner_picked = 0
    ork_id = 0
    if action == "up":
        new_state_location_gandolf[0] += 1
        if table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != '.' and table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != 'RPP' and table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != 'RPFP':
            already_in_danger = True
            ork_id = table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][1]
        if already_in_danger == False :
            max_danger = cur_state.get_max_danger()
        else :
            if cur_state.get_already_in_danger() :
                if cur_state.get_ork_id() == table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][1]:
                    max_danger = cur_state.get_max_danger() -1
            else :
                max_danger = table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0]
        if cur_state.get_picked() == True :
            if [new_state_location_gandolf[0],new_state_location_gandolf[1]] in Partners_Final and Partners_Final.index([new_state_location_gandolf[0],new_state_location_gandolf[1]]) == cur_state.partner_picked:
                picked = False
           
                new_state_goal_pos[cur_state.partner_picked] = 1
            else :
                picked = True
                partner_picked = cur_state.partner_picked
        else :
            if [new_state_location_gandolf[0],new_state_location_gandolf[1]] in Ring_Partners and new_state_goal_pos[Ring_Partners.index([new_state_location_gandolf[0],new_state_location_gandolf[1]])] == 0:
                picked = True
                partner_picked = Ring_Partners.index([new_state_location_gandolf[0],new_state_location_gandolf[1]])
              
            else :
                picked = False        
        new_state = State(new_state_location_gandolf, new_state_location_partners, already_in_danger,max_danger,new_state_goal_pos,picked,partner_picked,ork_id)
        return new_state
    elif action == "down":
        new_state_location_gandolf[0] -= 1
        if table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != '.' and table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != 'RPP' and table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != 'RPFP':
            already_in_danger = True
            ork_id = table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][1]
        if already_in_danger == False :
            max_danger = cur_state.get_max_danger()
        else :
            if cur_state.get_already_in_danger() :
                if cur_state.get_ork_id() == table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][1]:
                    max_danger = cur_state.get_max_danger() -1
            else :
                max_danger = table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0]
        if cur_state.get_picked() == True :
            if [new_state_location_gandolf[0],new_state_location_gandolf[1]] in Partners_Final and Partners_Final.index([new_state_location_gandolf[0],new_state_location_gandolf[1]]) == cur_state.partner_picked:
                picked = False
             
                new_state_goal_pos[cur_state.partner_picked] = 1
            else :
                picked = True
                partner_picked = cur_state.partner_picked
        else :
            if [new_state_location_gandolf[0],new_state_location_gandolf[1]] in Ring_Partners and new_state_goal_pos[Ring_Partners.index([new_state_location_gandolf[0],new_state_location_gandolf[1]])] == 0:
                picked = True
                partner_picked = Ring_Partners.index([new_state_location_gandolf[0],new_state_location_gandolf[1]])
            
            else :
                picked = False        
        new_state = State(new_state_location_gandolf, new_state_location_partners, already_in_danger,max_danger,new_state_goal_pos,picked,partner_picked,ork_id)

        return new_state
    elif action == "left":
        new_state_location_gandolf[1] -= 1
        if table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != '.' and table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != 'RPP' and table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != 'RPFP':
            already_in_danger = True
            ork_id = table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][1]
        if already_in_danger == False :
            max_danger = cur_state.get_max_danger()
        else :
            if cur_state.get_already_in_danger() :
                if cur_state.get_ork_id() == table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][1]:
                    max_danger = cur_state.get_max_danger() -1
            else :
                max_danger = table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0]
        if cur_state.get_picked() == True :
            if [new_state_location_gandolf[0],new_state_location_gandolf[1]] in Partners_Final and Partners_Final.index([new_state_location_gandolf[0],new_state_location_gandolf[1]]) == cur_state.partner_picked:
                picked = False
          
                new_state_goal_pos[cur_state.partner_picked] = 1
            else :
                picked = True
                partner_picked = cur_state.partner_picked
        else :
            if [new_state_location_gandolf[0],new_state_location_gandolf[1]] in Ring_Partners and new_state_goal_pos[Ring_Partners.index([new_state_location_gandolf[0],new_state_location_gandolf[1]])] == 0:
                picked = True
                partner_picked = Ring_Partners.index([new_state_location_gandolf[0],new_state_location_gandolf[1]])
     
            else :
                picked = False        
        new_state = State(new_state_location_gandolf, new_state_location_partners, already_in_danger,max_danger,new_state_goal_pos,picked,partner_picked,ork_id)

        return new_state
    elif action == "right":
        new_state_location_gandolf[1] += 1
        if table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != '.' and table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != 'RPP' and table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0] != 'RPFP':
            already_in_danger = True
            ork_id = table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][1]
        if already_in_danger == False :
            max_danger = cur_state.get_max_danger()
        else :
            if cur_state.get_already_in_danger() :
                if cur_state.get_ork_id() == table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][1]:
                    max_danger = cur_state.get_max_danger() -1
            else :
                max_danger = table[new_state_location_gandolf[0]][new_state_location_gandolf[1]][0]
        if cur_state.get_picked() == True :
            if [new_state_location_gandolf[0],new_state_location_gandolf[1]] in Partners_Final and Partners_Final.index([new_state_location_gandolf[0],new_state_location_gandolf[1]]) == cur_state.partner_picked:
                picked = False
   
                new_state_goal_pos[cur_state.partner_picked] = 1
            else :
                picked = True
                partner_picked = cur_state.partner_picked
        else :
            if [new_state_location_gandolf[0],new_state_location_gandolf[1]] in Ring_Partners and new_state_goal_pos[Ring_Partners.index([new_state_location_gandolf[0],new_state_location_gandolf[1]])] == 0:
                picked = True
                partner_picked = Ring_Partners.index([new_state_location_gandolf[0],new_state_location_gandolf[1]])

            else :
                picked = False        
        new_state = State(new_state_location_gandolf, new_state_location_partners, already_in_danger,max_danger,new_state_goal_pos,picked,partner_picked,ork_id)

        return new_state

## BFS algorithm :

"For recognizing final state there is a method in class above as finish_condition which it called after making each new state and if the new state satisfies all the conditions (all the partners have been arrived to their goal positions and Gandolf is in the last cell of map) then this methode will return true and then searching is over."

In [68]:
def print_path(last_state_str):
    path = list()
    path.append(last_state_str.split("-")[0][:-int(l)-2])
    while last_state_str != first_state.represent_as_str_with_id():
        state_str = father[last_state_str].split("-")[0]
        state_str_locations = state_str[:-int(l)-2]
        path.append(state_str_locations)
        last_state_str = father[last_state_str]
    print(f"number of moves : {len(path)-1}")
    print("path is :")
    print(list(reversed(path)))

def BFS():
    frontier = deque()
    frontier.append(first_state)
    states_seen = 1
    explored = {}
    explored[first_state.represent_as_str()] = 1
    while True:

        if len(frontier) == 0:
            return "failure"

        cur_state = frontier.popleft()
        if [cur_state.get_row(), cur_state.get_column()] == [int(xn) , int(yn)]:
            if sum(cur_state.get_goal_pos()) == int(l):
                return "done"
        for action in actions:

            if allowable_action(cur_state, action):
                child_state = transition_func(cur_state, action)
                if child_state.represent_as_str() not in explored:
                    states_seen += 1
                    explored[child_state.represent_as_str()] = 1
                    father[child_state.represent_as_str_with_id()] = cur_state.represent_as_str_with_id()
                    frontier.append(child_state)

                    if child_state.finish_condition():
                        print(f"number of states seen is : {states_seen}")
                        print_path(child_state.represent_as_str_with_id())
                        return "problem solved"
start_bfs = time.time()
print(BFS())
finish_bfs = time.time()
print(f"time consumed : {finish_bfs - start_bfs}")

number of states seen is : 416
number of moves : 32
path is :
['00', '01', '02', '03', '04', '05', '06', '07', '17', '27', '37', '47', '57', '56', '55', '54', '53', '52', '51', '50', '60', '70', '71', '72', '73', '74', '75', '76', '66', '56', '66', '76', '77']
problem solved
time consumed : 0.012965202331542969


## BFS result table : 

| test number | length of optimal path | number of states seen | average time consumed |
| --- | --- | --- | --- |
| test 1 | 48 | 7329 | 0.22 s |
| test 2 | 52 | 1381 | 0.04 s |
| test 3 | 32 | 416 | 0.01 s |
| test 4 | 66 | 4310 | 0.13 s |



Advantages :

 The solution will definitely found out by BFS If there is some solution.
 BFS will never get trapped in a blind alley, which means unwanted nodes.
 If there is more than one solution then it will find a solution with minimal steps.

Disadvantages :

 Memory Constraints As it stores all the nodes of the present level to go for the next level.
 If a solution is far away then it consumes time.

## IDS algorithm:

In [69]:
states_seen = 0
explored = {}
def DLS(cur_state, max_depth ):
    global states_seen 
    states_seen += 1
    if cur_state.finish_condition():
        print_path(cur_state.represent_as_str_with_id())
        return True
    elif max_depth == 0:
        return False

    if [cur_state.get_row(), cur_state.get_column()] == [int(xn) , int(yn)]:
            if sum(cur_state.get_goal_pos()) == int(l):
                return "done"
    for action in actions:
        if allowable_action(cur_state, action):
            child_state = transition_func(cur_state, action)
            child_state.set_remained_depth(max_depth)
            if child_state.represent_as_str() not in explored or int(child_state.get_remained_depth()) > explored[child_state.represent_as_str()]:
    
                explored[child_state.represent_as_str()] = max_depth
                father[child_state.represent_as_str_with_id()] = cur_state.represent_as_str_with_id()

                if DLS(child_state, max_depth - 1):
                    return True
    return False


def IDS(max_depth):
    for depth in range(max_depth):
        explored.clear()
        father.clear()
        explored[first_state.represent_as_str()] = 1
        if DLS(first_state, depth):
            return "problem solved"
    return "failure"


start_ids = time.time()
print(IDS(100))
print(f"number of states seen is : {states_seen}")
finish_ids = time.time()
print(f"time consumed : {finish_ids - start_ids}")
print("--------------------------------")

number of moves : 32
path is :
['00', '01', '02', '03', '04', '05', '06', '07', '17', '27', '37', '47', '57', '56', '55', '54', '53', '52', '51', '50', '60', '70', '71', '72', '73', '74', '75', '76', '66', '56', '66', '76', '77']
problem solved
number of states seen is : 22722
time consumed : 0.7255465984344482
--------------------------------


## IDS result table : 

| test number | length of optimal path | number of states seen | average time consumed |
| --- | --- | --- | --- |
| test 1 | 48 | 137085 | 4.6 s |
| test 2 | 52 | 233643 | 7.9 s |
| test 3 | 32 | 22722 | 0.71 s |
| test 4 | 66 | 931438 | 33.80 s |


Advantages :
 When the solutions are found at the lower depths say n, then the algorithm proves to be efficient and in time.
 
Disadvantages :
 Iterative deepening search generates the states multiple times.
 The main problem with IDDFS is the time and wasted calculations that take place at each depth.
 The time taken is exponential to reach the goal node.

## A* algorithm :

 The heuristic we use in this part is Euclidean distance  which returns the value bellow for each state .

  $(\sqrt{(x_g - x_i)^2 + (y_g - y_i)^2})$

 which $x_g , y_g$ are the row and column of the last cell of map.

 As we can go only in 4 direction (up , down , left and right) and the real cost of each movement is 1 so the difference value of Euclidean distance of each new state from the previous state is 1 which satisfies the consistenty condition so this heuristic function is consistent .
 
 In the function A* bellow, data structure min heap is used in variable frontier so we can find minimum cost ( g(n) + alpha* h(n) ) of the states in frontier in O(1).



In [70]:
from heapq import heappop , heappush , heapify
visited = {}
exploredd = []
def A_star(alpha):
    frontier = []
    heapify(frontier)
    first_state_cost = float(first_state.represent_as_str_with_cost(alpha).split("-")[1])
    heappush(frontier, [first_state_cost, first_state])
    visited[first_state.represent_as_str()] = 1
    states_seen = 1
    exploredd.append(first_state.represent_as_str())
    while len(frontier) > 0:
        
        cur_state = heappop(frontier)[1]

        exploredd.append(cur_state.represent_as_str())
        if [cur_state.get_row(), cur_state.get_column()] == [int(xn) , int(yn)]:
            if sum(cur_state.get_goal_pos()) == int(l):
                return "done"

        for action in actions:

            if allowable_action(cur_state, action):
                child_state = transition_func(cur_state, action)
                child_state.set_distance(cur_state.get_distance() + 1)
            if child_state.represent_as_str() not in visited :
                states_seen += 1
                visited[child_state.represent_as_str()] = 1
                father[child_state.represent_as_str_with_id()] = cur_state.represent_as_str_with_id()
                child_state_cost = float(child_state.represent_as_str_with_cost(alpha).split("-")[1])
                heappush(frontier, [child_state_cost, child_state])

                if child_state.finish_condition():
                    print(f"number of states seen is : {states_seen}")
                    print_path(child_state.represent_as_str_with_id())
                    return "problem solved"


visited.clear()
father.clear()
exploredd.clear()
start_A_star = time.time()
print(A_star(alpha = 1))
finish_A_star = time.time()
print(f"time consumed : {finish_A_star - start_A_star}")
print("--------------------------------")

number of states seen is : 351
number of moves : 32
path is :
['00', '01', '02', '03', '04', '05', '06', '07', '17', '27', '37', '47', '57', '56', '55', '54', '53', '52', '51', '50', '51', '61', '71', '72', '73', '74', '75', '76', '66', '56', '66', '76', '77']
problem solved
time consumed : 0.013962745666503906
--------------------------------


## A* result table (alpha = 1) : 

| test number | length of optimal path | number of states seen | average time consumed |
| --- | --- | --- | --- |
| test 1 | 48 | 3480 | 0.14 s |
| test 2 | 52 | 1306 | 0.06 s |
| test 3 | 32 | 351 | 0.01 s |
| test 4 | 66 | 4113 | 0.17 s |



Advantages :
 It is complete and optimal.
 It is optimally efficient, i.e. there is no other optimal algorithm guaranteed to expand fewer nodes than A*.
 It is the best one from other techniques. It is used to solve very complex problems.
Disadvantages :
 The speed execution of A* search is highly dependant on the accuracy of the heuristic algorithm that is used to compute h (n).

## Weighted A* algorithm with alpha = 2 :
 In this part we change the cost of each state to cost = g(n) + alpha * h(n)


In [71]:
visited.clear()
father.clear()
exploredd.clear()
start_A_star = time.time()
print(A_star(alpha = 2))
finish_A_star = time.time()
print(f"time consumed : {finish_A_star - start_A_star}")
print("--------------------------------")

number of states seen is : 324
number of moves : 36
path is :
['00', '10', '11', '21', '22', '23', '24', '25', '15', '16', '17', '07', '17', '27', '37', '47', '57', '56', '55', '54', '53', '52', '51', '50', '51', '61', '71', '72', '73', '74', '75', '76', '66', '56', '66', '76', '77']
problem solved
time consumed : 0.012964248657226562
--------------------------------


## A* result table (alpha = 2) : 

| test number | length of path | number of states seen | average time consumed |
| --- | --- | --- | --- |
| test 1 | 48 | 2665 | 0.10 s |
| test 2 | 54 | 1214 | 0.05 s |
| test 3 | 36 | 324 | 0.01 s |
| test 4 | 68 | 3836 | 0.16 s |

 As it can be seen the A* algorithm with alpha = 2 is faster and has seen less states than with alpha = 1 but the path is not shortest path(optimal) and its longer than A* with alpha = 1.

## Weighted A* algorithm with alpha = 3 :

In [72]:
visited.clear()
father.clear()
exploredd.clear()
start_A_star = time.time()
print(A_star(alpha = 3))
finish_A_star = time.time()
print(f"time consumed : {finish_A_star - start_A_star}")
print("--------------------------------")

number of states seen is : 282
number of moves : 36
path is :
['00', '10', '11', '21', '22', '23', '24', '14', '15', '16', '17', '07', '17', '27', '37', '47', '57', '56', '55', '54', '53', '52', '51', '50', '51', '61', '71', '72', '73', '74', '75', '76', '66', '56', '66', '76', '77']
problem solved
time consumed : 0.011032581329345703
--------------------------------


## A* result table (alpha = 3) : 

| test number | length of path | number of states seen | average time consumed |
| --- | --- | --- | --- |
| test 1 | 48 | 1308 | 0.05 s |
| test 2 | 56 | 1114 | 0.04 s |
| test 3 | 36 | 282 | 0.009 s |
| test 4 | 70 | 3621 | 0.15 s |


 As it can be seen the A* algorithm with alpha = 3 is faster and has seen less states than with alpha = 2 but the path is not shortest path(optimal) and its longer than A* with alpha = 2.
